In [2]:
import pandas as pd
import os 

In [3]:
os.chdir('C:\\Users\\acer\\OneDrive\\Desktop\\Project_Git\\CA1-DAAA2B03-2423542-ChowKwokYao')
df = pd.read_csv('AmesHousing.csv')

In [4]:
df

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,...,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,...,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,170000


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# ==========================================
# 1. LOAD DATA & INITIAL CONFIG
# ==========================================
# Load the dataset (ensure the file is in the same folder)
df = pd.read_csv('AmesHousing.csv')

# Standardize column names: Remove spaces for easier coding
# Example: 'Lot Frontage' becomes 'LotFrontage'
df.columns = [c.replace(' ', '') for c in df.columns]

print(f"Original Shape: {df.shape}")

# ==========================================
# 2. DROP IRRELEVANT COLUMNS
# ==========================================
# 'Order' and 'PID' are just indexers/ID numbers, not useful for prediction.
df.drop(['Order', 'PID'], axis=1, inplace=True)

# ==========================================
# 3. HANDLING "FAKE" MISSING VALUES (Categorical)
# ==========================================
# Documentation: In the Ames dataset, NA in these columns doesn't mean "Missing".
# It means the feature does not exist (e.g., No Pool, No Garage).
# We fill these with the string "None" to treat it as a valid category.

cols_fill_none = [
    'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'
]

for col in cols_fill_none:
    if col in df.columns:
        df[col] = df[col].fillna("None")

# ==========================================
# 4. HANDLING "FAKE" MISSING VALUES (Numerical)
# ==========================================
# Documentation: If a house has no Garage, the GarageArea is 0, not the median.
# We fill these NAs with 0.

cols_fill_zero = [
    'GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1',
    'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath',
    'BsmtHalfBath', 'MasVnrArea'
]

for col in cols_fill_zero:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# ==========================================
# 5. HANDLING REAL MISSING VALUES
# ==========================================
# 'LotFrontage': Linear feet of street connected to property.
# We fill with Median because it is a physical measurement (robust to outliers).
if 'LotFrontage' in df.columns:
    df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())

# 'MasVnrType' and 'Electrical': Minimal missing rows.
# We fill with the Mode (most common value).
if 'MasVnrType' in df.columns:
    df['MasVnrType'] = df['MasVnrType'].fillna(df['MasVnrType'].mode()[0])
if 'Electrical' in df.columns:
    df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

# ==========================================
# 6. FEATURE ENGINEERING: ORDINAL ENCODING
# ==========================================
# Documentation: Many quality columns use a scale (Ex, Gd, Ta, Fa, Po).
# Standard One-Hot Encoding would lose the "order" (that Ex > Gd).
# We map these to numbers 5, 4, 3, 2, 1 manually to help the model.

quality_map = {'Ex': 5, 'Gd': 4, 'Ta': 3, 'Fa': 2, 'Po': 1, 'None': 0}

quality_cols = [
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
    'HeatingQC', 'KitchenQual', 'FireplaceQu',
    'GarageQual', 'GarageCond', 'PoolQC'
]

for col in quality_cols:
    if col in df.columns:
        df[col] = df[col].map(quality_map)
        # Fill any new NaNs (unexpected values) with 0
        df[col] = df[col].fillna(0)

# ==========================================
# 7. FEATURE SELECTION FOR WEB APP
# ==========================================
# We only want the most important features for our Flask App.
# Calculate correlation with SalePrice
corr_matrix = df.select_dtypes(include=[np.number]).corr()
top_features = corr_matrix['SalePrice'].sort_values(ascending=False).head(10).index.tolist()

print(f"Top 10 Features selected for Web App: {top_features}")

# ==========================================
# 8. SAVE CLEANED DATA
# ==========================================
# Save the full cleaned dataset (for training the complex model)
df.to_csv('AmesHousing_Cleaned.csv', index=False)

# Save the subset (just the top columns needed for the Web App)
df_subset = df[top_features].copy()
df_subset.to_csv('AmesHousing_WebApp_Subset.csv', index=False)

print("Success! 'AmesHousing_Cleaned.csv' and 'AmesHousing_WebApp_Subset.csv' have been saved.")

Original Shape: (2930, 82)
Top 10 Features selected for Web App: ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'ExterQual', 'TotalBsmtSF', '1stFlrSF', 'KitchenQual', 'BsmtQual']
Success! 'AmesHousing_Cleaned.csv' and 'AmesHousing_WebApp_Subset.csv' have been saved.


In [6]:
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RL,141.0,31770,Pave,None,IR1,Lvl,AllPub,Corner,...,0,0.0,None,None,0,5,2010,WD,Normal,215000
1,20,RH,80.0,11622,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0.0,MnPrv,None,0,6,2010,WD,Normal,105000
2,20,RL,81.0,14267,Pave,None,IR1,Lvl,AllPub,Corner,...,0,0.0,None,Gar2,12500,6,2010,WD,Normal,172000
3,20,RL,93.0,11160,Pave,None,Reg,Lvl,AllPub,Corner,...,0,0.0,None,None,0,4,2010,WD,Normal,244000
4,60,RL,74.0,13830,Pave,None,IR1,Lvl,AllPub,Inside,...,0,0.0,MnPrv,None,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,80,RL,37.0,7937,Pave,None,IR1,Lvl,AllPub,CulDSac,...,0,0.0,GdPrv,None,0,3,2006,WD,Normal,142500
2926,20,RL,68.0,8885,Pave,None,IR1,Low,AllPub,Inside,...,0,0.0,MnPrv,None,0,6,2006,WD,Normal,131000
2927,85,RL,62.0,10441,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0.0,MnPrv,Shed,700,7,2006,WD,Normal,132000
2928,20,RL,77.0,10010,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0.0,None,None,0,4,2006,WD,Normal,170000
